In [1]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# 데이터 불러오기

In [2]:
df = pd.read_csv('0507 df_for_holdout.csv')

In [4]:
df.set_index('Date',inplace = True)

In [6]:
df.head()

,시언감성점수,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,시언서술포함감성점수,주택,아파트,분양,서울,가구,부동산,단지,시장,next_spi
Date,,,,,,,,,,,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,-1.635618e+08,1160,1094,666,729,1180,685,461,459,82.5
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,-1.777621e+08,1160,1101,639,652,1265,669,383,496,83.4
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,-1.980306e+08,1341,1102,1014,607,961,659,576,578,83.9
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,-1.898849e+08,1405,1186,977,658,1031,720,534,577,84.1
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,-2.402702e+08,1537,1543,1267,854,1245,1013,770,756,84.4


# Train/Test Set 분할

In [7]:
X = df.iloc[:,:-1]
y = df['next_spi'].to_list()

In [8]:
X.head()

,시언감성점수,M2,comp basemoney rate,base rate,consumer price index,kospi close,ex-rate close,효준감성점수,시언서술포함감성점수,주택,아파트,분양,서울,가구,부동산,단지,시장
Date,,,,,,,,,,,,,,,,,
201101월,-6.268386e+07,1676.4,4.52,2.75,3.4,2069.73,1119.25,1856.746125,-1.635618e+08,1160,1094,666,729,1180,685,461,459
201102월,-6.255832e+07,1674.4,4.72,2.75,3.9,1939.30,1124.65,2046.702906,-1.777621e+08,1160,1101,639,652,1265,669,383,496
201103월,-6.957441e+07,1677.5,4.54,3.00,4.1,2106.70,1095.50,1715.795289,-1.980306e+08,1341,1102,1014,607,961,659,576,578
201104월,-7.305733e+07,1684.8,4.54,3.00,3.8,2192.36,1068.00,1635.262791,-1.898849e+08,1405,1186,977,658,1031,720,534,577
201105월,-9.356633e+07,1690.5,4.44,3.00,3.9,2142.47,1078.00,1922.518709,-2.402702e+08,1537,1543,1267,854,1245,1013,770,756


In [11]:
y[:5]

[82.5, 83.4, 83.9, 84.1, 84.4]

In [12]:
# Holdout으로 Train/Test 분할
X_train = X.iloc[:90]
y_train = y[:90]
X_test = X.iloc[90:]
y_test = y[90:]

In [15]:
X_train.columns

Index(['시언감성점수', 'M2', 'comp basemoney rate', 'base rate',
       'consumer price index', 'kospi close', 'ex-rate close', '효준감성점수',
       '시언서술포함감성점수', '주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],
      dtype='object')

tf_idf_words = ['주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장']

# 거시경제변수만 포함된 Case

In [16]:
# 거시경제변수만 포함된 X 만들기
X_train_basic = X_train.drop(['시언감성점수','효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'], axis = 1)
X_test_basic = X_test.drop(['시언감성점수','효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'], axis = 1)

In [17]:
# SVM 인풋 데이터 스케일링
sc = StandardScaler()
X_train_basic_svm = sc.fit_transform(X_train_basic)
X_test_basic_svm = sc.transform(X_test_basic)

In [18]:
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid_svm = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf','sigmoid']} 
  
grid_svm = GridSearchCV(SVR(), param_grid_svm, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_svm.fit(X_train_basic_svm, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             refit='mse')

In [19]:
y_pred = grid_svm.predict(X_test_basic_svm)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_svm.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_svm.best_estimator_)

8.350425444114785
{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
SVR(C=1000, gamma=0.01)


In [20]:
del param_grid_svm, grid_svm, RMSE, y_pred

# 시언 감성 점수 Case

## 시언 감성 점수 데이터 전처리

In [21]:
# 시언 감성 점수 (명사만 이용) 포함 X 만들기
X_train_si = X_train.drop(['효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_test_si = X_test.drop(['효준감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)

In [22]:
X_train_si_verb = X_train.drop(['시언감성점수','효준감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_test_si_verb = X_test.drop(['시언감성점수','효준감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)

In [23]:
# SVM용 인풋 데이터 스케일링
sc = StandardScaler()

# 시언 감성 점수 (명사만 이용) => 표준화 
X_train_si_svm = sc.fit_transform(X_train_si)
X_test_si_svm = sc.transform(X_test_si)

# 시언 감성 점수 (서술어 포함) => 표준화
X_train_si_verb_svm = sc.fit_transform(X_train_si_verb)
X_test_si_verb_svm = sc.transform(X_test_si_verb)

## (시언 감성 점수 (명사)) 이용 SVR 수립 및 평가

In [24]:
# defining parameter range
param_grid_svm = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf','sigmoid']} 
  
grid_svm = GridSearchCV(SVR(), param_grid_svm, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_svm.fit(X_train_si_svm, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             refit='mse')

In [25]:
y_pred = grid_svm.predict(X_test_si_svm)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_svm.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_svm.best_estimator_)

5.532520407345845
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVR(C=10, gamma=0.01)


In [26]:
del param_grid_svm, grid_svm, RMSE, y_pred

## (시언 감성 점수 (서술어 포함)) 이용 SVR 수립 및 평가

In [27]:
# defining parameter range
param_grid_svm = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf','sigmoid']} 
  
grid_svm = GridSearchCV(SVR(), param_grid_svm, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_svm.fit(X_train_si_verb_svm, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             refit='mse')

In [28]:
y_pred = grid_svm.predict(X_test_si_svm)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_svm.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_svm.best_estimator_)

15.661205389881301
{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
SVR(C=10, gamma=0.01)


In [29]:
del param_grid_svm, grid_svm, RMSE, y_pred

# 효준 감성 점수(명사) Case

## 효준 감성 점수 포함 성능

In [30]:
X_train_hj = X_train.drop(['시언감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)
X_test_hj = X_test.drop(['시언감성점수','시언서술포함감성점수','주택', '아파트', '분양', '서울', '가구', '부동산', '단지', '시장'],axis = 1)

In [31]:
sc = StandardScaler()
X_train_hj = sc.fit_transform(X_train_hj)
X_test_hj = sc.transform(X_test_hj)

In [32]:
# defining parameter range
param_grid_svm = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf','sigmoid']} 
  
grid_svm = GridSearchCV(SVR(), param_grid_svm, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_svm.fit(X_train_hj, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             refit='mse')

In [33]:
y_pred = grid_svm.predict(X_test_hj)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_svm.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_svm.best_estimator_)

7.9575509904832975
{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
SVR(C=1000, gamma=0.01)


In [34]:
del param_grid_svm, grid_svm, RMSE, y_pred

# TF-IDF 상위 8개 단어 빈도수 추가 Case

In [35]:
X_train_tf_idf = X_train.drop(columns = [], axis = 1)
X_test_tf_idf = X_test.drop(columns = [], axis = 1)

In [36]:
sc = StandardScaler()
X_train_tf_idf = sc.fit_transform(X_train_tf_idf)
X_test_tf_idf = sc.transform(X_test_tf_idf)

In [37]:
# defining parameter range
param_grid_svm = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear','poly','rbf','sigmoid']} 
  
grid_svm = GridSearchCV(SVR(), param_grid_svm, refit = 'mse', verbose = 0, cv = 5)

# fitting the model for grid search
grid_svm.fit(X_train_tf_idf, y_train)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             refit='mse')

In [38]:
y_pred = grid_svm.predict(X_test_tf_idf)
RMSE = mean_squared_error(y_test, y_pred)**0.5

# RMSE 출력
print(RMSE)

# print best parameter after tuning
print(grid_svm.best_params_)
  
# print how our model looks after hyper-parameter tuning
print(grid_svm.best_estimator_)

7.07880868749114
{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
SVR(C=100, gamma=0.01)


In [39]:
del param_grid_svm, grid_svm, RMSE, y_pred